## Seach $\mu$ (minimal mutual information)

In [6]:
"""
    search_optimal_aug_params
"""
import os
os.chdir("/home1/quanquan/code/landmark/code/tproj/")
import cv2
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
# from scipy.misc import imread
import numpy as np
import cv2
from skimage.exposure import histogram
from scipy.stats import entropy
import matplotlib
from tutils import tfilename
from PIL import Image
import torchvision.transforms.functional as F
from scipy.stats import norm
matplotlib.use('Agg')  # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
plt.style.use('ggplot')

EX_CONFIG = {
    'dataset': {
        'name': 'Cephalometric',
        'pth': '/home1/quanquan/datasets/Cephalometric/',
        'entr': '/home1/quanquan/datasets/Cephalometric/entr1/train/',
        'prob': '/home1/quanquan/datasets/Cephalometric/prob/train/',
        'n_cls': 19,
    }
}

def test_all(landmark_list, landmark_id=0):
    from torchvision import transforms
    from PIL import Image
    from tutils import tfilename

    pth = EX_CONFIG['dataset']['pth']
    if landmark_list is None:
        from datasets.ceph.ceph_ssl import Test_Cephalometric
        testset = Test_Cephalometric(pth, mode="Train")
        # item, landmark_list, template_patches = testset.__getitem__(0)
        # data = testset.__getitem__(id_oneshot)
        landmark_list = []
        for i in range(len(testset)):
            landmark = testset.ref_landmarks(i)[landmark_id]
            landmark_list.append(landmark)

    def get_fea(patch):
        fea = np.zeros((256,))
        hist, idx = histogram(patch, nbins=256)
        for hi, idi in zip(hist, idx):
            # print(hi, idi, i, j)
            fea[idi] = hi
        return fea

    im_list = []
    fea_list = []
    # landmark_id = 3
    for i in range(150):
        # im_pth = tfilename(pth, "RawImage/TrainingData", f"{i+1:03d}.bmp")
        im_pth = tfilename(pth, "RawImage/NoisyTrainingData75", f"{i+1:03d}.bmp")
        # print(im_pth)
        im = cv2.imread(im_pth, cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (384, 384))
        lm = landmark_list[i]
        ps_half = 32
        patch = im[max(lm[0]-ps_half, 0):lm[0]+ps_half, max(lm[1]-ps_half, 0):lm[1]+ps_half]
        if not (patch.shape[0] > 0 and patch.shape[1] > 0):
            import ipdb; ipdb.set_trace()
        # assert patch.shape == (64, 64), f"Got {patch.shape}"
        fea1 = get_fea(patch)
        fea_list.append(fea1)

    fea0 = fea_list[114]
    # mi0 = mutual_info_score(fea0, fea0)
    mi_list = []
    for i in range(150):
        mii = mutual_info_score(fea0, fea_list[i])
        mi_list.append(mii)
    # print(mi_list)
    mi_list = np.array(mi_list)
    print("max mi: ", mi_list.max())
    print("min mi: ",  mi_list.min())
    # import ipdb; ipdb.set_trace()
    # draw_bar_auto_split(mi_list)
    # return mi_list.max(), mi_list.min(), mi_list.mean()
    return mi_list

def ana_test_all():
    res_list = []
    for i in range(19):
        res = test_all(None, i)
        # print("---------------------------")
        # print(res)
        # print("---------------------------")
        res_list.append(res)
        # import ipdb; ipdb.set_trace()
    res_total = np.concatenate(res_list, axis=0)
    print("res total ", res_total.mean(), res_total.std()) # ceph: mean:1.557; std:0.4429 # ceph: mean: 1.4925002126243447 std: 0.3928087723258639

    # draw_bar_auto_split(res_total, tag=99)
    # np.save(tfilename("./cache/res_total_lm.npy"), res_total)


In [7]:
ana_test_all()

The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  2.9768203987342887
min mi:  0.8455594107236846
The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  3.478368118468378
min mi:  0.8909906546325643
The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  3.455271261551705
min mi:  1.3368848730722867
The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  2.3198318705211416
min mi:  0.24701133950822438
The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  3.2056270723603584
min mi:  0.9116811242192856
The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  2.7441647184551643
min mi:  0.7074892629929762
The sizes are set as  [384, 384]
Initializing Datasets: (split:'Train') (len:(150)) 
max mi:  0.9914967128900943
min mi:  0.3303490212069548
The sizes are 